In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas import DataFrame 
import nltk

from sklearn.neighbors import NearestNeighbors
from sklearn.linear_model import LogisticRegression
from sklearn import neighbors
from scipy.spatial.distance import cosine
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectKBest
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

import re
import string
from sklearn.metrics import mean_squared_error

In [2]:
df = pd.read_json("Office_Products_5.json", lines=True)
print(df.columns)
print(df.shape)

Index(['asin', 'helpful', 'overall', 'reviewText', 'reviewTime', 'reviewerID',
       'reviewerName', 'summary', 'unixReviewTime'],
      dtype='object')
(53258, 9)


In [3]:
df = df.sort_values(by='asin')
df = df.reset_index(drop=True)
count = df.groupby("asin", as_index=False).mean()
count_users = df.groupby("reviewerID", as_index=False).count()

df_clean = df[['asin', 'reviewerID', 'overall']]
print(df_clean)

             asin      reviewerID  overall
0      B00000JBLH  A32T2H8150OJLU        5
1      B00000JBLH  A3MAFS04ZABRGO        5
2      B00000JBLH  A1F1A0QQP2XVH5        2
3      B00000JBLH   A49R5DBXXQDE5        5
4      B00000JBLH  A2XRMQA6PJ5ZJ8        5
5      B00000JBLH  A2JFOHC9W629IE        5
6      B00000JBLH  A38NELQT98S4H8        5
7      B00000JBLH   AA8M6331NI1EN        5
8      B00000JBLU  A1LYT85S93HUNY        5
9      B00000JBLU   A4XCTN5YLC89U        5
10     B00000JBLU  A2A1XYSB692L6J        4
11     B00000JBLU   AGBFIQMV9L61Y        5
12     B00000JBLU  A2R6RA8FRBS608        5
13     B00000JBLU  A30U2QQN2FFHE9        5
14     B00000JBLU  A1RTVWTWZSIC94        5
15     B00000JBLU  A25C2M3QF9G7OQ        5
16     B00000JBLU  A2W0XCWOK3MW6F        5
17     B00000JBNX   A6KL17KKN0A5L        5
18     B00000JBNX  A3LZA698SQPCXE        5
19     B00000JBNX   AO9X6644SGODF        5
20     B00000JBNX   AOUM75FA941HX        2
21     B00000JBNX  A36OT34BWUV59E        5
22     B000

In [4]:
n_users = df_clean.reviewerID.unique().shape[0]
n_items = df_clean.asin.unique().shape[0]
print ('Number of movies = ' + str(n_items)+' | Number of users = ' + str(n_users))

Number of movies = 2420 | Number of users = 4905


In [5]:
count_users.loc[count_users['reviewerID'] == 'AZ9WQM7SLUX7E']

,reviewerID,asin,helpful,overall,reviewText,reviewTime,reviewerName,summary,unixReviewTime
4877,AZ9WQM7SLUX7E,21,21,21,21,21,20,21,21


In [6]:
items_df = count[['asin']]
items_df
users_df = count_users[['reviewerID']]
users_df
users_list = users_df.values
users_list

array([['A00473363TJ8YSZ3YAGG9'],
       ['A04324702R5O7JRSQNVAH'],
       ['A0678500JUN3N7KUG4PS'],
       ...,
       ['AZYJE40XW6MFG'],
       ['AZZ5ASC403N74'],
       ['AZZD30PYJVGI7']], dtype=object)

In [7]:
from sklearn.model_selection import train_test_split
trainSet, testSet = train_test_split(df_clean, test_size=0.2,stratify=df_clean['reviewerID'])

print(len(testSet))
print(len(trainSet))

print(testSet.shape)
print(trainSet.shape)


10652
42606
(10652, 3)
(42606, 3)


In [8]:
mean_items = trainSet.groupby(['asin']).mean()

mean_users = trainSet.groupby(['reviewerID']).mean()

mean_overall = trainSet["overall"].mean()

print(mean_items,mean_users,mean_overall)

             overall
asin                
B00000JBLH  4.571429
B00000JBLU  4.857143
B00000JBNX  4.333333
B00000JBO8  4.000000
B00000JFNV  4.757576
B00000JZKB  4.000000
B00000K3T0  5.000000
B00001MXVQ  3.333333
B00002NDS2  4.785714
B00002QUKW  5.000000
B000031WDB  3.846154
B000034DLQ  4.142857
B0000365QL  4.666667
B00003CYNU  5.000000
B00004SDB1  4.400000
B00004SUJT  5.000000
B00004TS2I  5.000000
B00004TS2L  4.857143
B00004U8RE  4.700000
B00004VVIX  4.300000
B00004YTPX  4.200000
B00004YV1W  4.142857
B00004Z47L  5.000000
B00004Z498  4.705882
B00004Z4A8  4.666667
B00004Z4BV  4.888889
B00004Z59A  4.190476
B00004Z5DW  4.142857
B00004Z5QI  4.285714
B00004Z5QO  4.719298
...              ...
B00J8NBVYE  4.444444
B00J8U9QCG  4.133333
B00J9E68B8  4.000000
B00JAKXFQW  4.590909
B00JBK74IQ  4.358974
B00JBVWDB8  4.800000
B00JCA4GQS  4.571429
B00JCA4GW2  4.666667
B00JHME7W4  4.000000
B00JIX4OTI  4.545455
B00JIX53PW  5.000000
B00JIXXHEG  4.200000
B00JJRMPPS  4.615385
B00JJS7JXK  3.566667
B00JKQV90A  3

In [9]:
trainSet

,asin,reviewerID,overall
20944,B002FGUDBS,ATFBVUXDIRXT6,5
38024,B005IMAKYG,A3H7NPNDMGLOU4,5
13315,B0010JEJPC,AMW1OYXWOGZGJ,5
41923,B0085IQ28I,A15T9G38F589KM,2
44729,B009ZMWHIC,A32E5G5T902WRV,1
46841,B00BV1XGGM,ADLVFFE4VBT8,4
33841,B004INGE4G,A2UQIQUOAB47OT,5
31772,B00466DTE8,A1I477ADGMLVJM,3
14954,B0015G39OW,A2FZQF0MH29VYN,4
47608,B00CPXCVIO,A194K60CWZ371J,3


In [10]:
# print(trainSet.groupby("asin", as_index=False).count())

In [11]:
train_data_matrix_na = trainSet.pivot(index='asin', columns='reviewerID', values='overall')
train_data_matrix = train_data_matrix_na
train_data_matrix

reviewerID,A00473363TJ8YSZ3YAGG9,A04324702R5O7JRSQNVAH,A0678500JUN3N7KUG4PS,A100UD67AHFODS,A100WO06OQR8BQ,A100ZQDV7L8PVV,A101PIL0I3UZJJ,A1027EV8A9PV1O,A1047EDJ84IMAS,A105S56ODHGJEK,...,AZMY6E8B52L2T,AZNUHQSHZHSUE,AZO8OJ7YGSJSS,AZQAEL5GPBNIR,AZQGJ5CEAJGXB,AZQJ85BTRUQV2,AZU2JDR2GVICQ,AZYJE40XW6MFG,AZZ5ASC403N74,AZZD30PYJVGI7
asin,,,,,,,,,,,,,,,,,,,,,
B00000JBLH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B00000JBLU,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B00000JBNX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B00000JBO8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B00000JFNV,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B00000JZKB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B00000K3T0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B00001MXVQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B00002NDS2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
train_values = train_data_matrix.values
train_sum = train_data_matrix.sum(axis = 1)
train_count = train_data_matrix.count(axis =1).astype('float64')
train_normal = train_sum.div(train_count,axis=0)

train_data_matrix = train_data_matrix.sub(train_normal,axis=0).fillna(0)
train_data_matrix

reviewerID,A00473363TJ8YSZ3YAGG9,A04324702R5O7JRSQNVAH,A0678500JUN3N7KUG4PS,A100UD67AHFODS,A100WO06OQR8BQ,A100ZQDV7L8PVV,A101PIL0I3UZJJ,A1027EV8A9PV1O,A1047EDJ84IMAS,A105S56ODHGJEK,...,AZMY6E8B52L2T,AZNUHQSHZHSUE,AZO8OJ7YGSJSS,AZQAEL5GPBNIR,AZQGJ5CEAJGXB,AZQJ85BTRUQV2,AZU2JDR2GVICQ,AZYJE40XW6MFG,AZZ5ASC403N74,AZZD30PYJVGI7
asin,,,,,,,,,,,,,,,,,,,,,
B00000JBLH,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B00000JBLU,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B00000JBNX,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B00000JBO8,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B00000JFNV,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B00000JZKB,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B00000K3T0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B00001MXVQ,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B00002NDS2,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
test_data_matrix_na = testSet.pivot(index='asin', columns='reviewerID', values='overall')
test_data_matrix = test_data_matrix_na.fillna(0)
test_data_matrix

reviewerID,A00473363TJ8YSZ3YAGG9,A04324702R5O7JRSQNVAH,A0678500JUN3N7KUG4PS,A100UD67AHFODS,A100WO06OQR8BQ,A100ZQDV7L8PVV,A101PIL0I3UZJJ,A1027EV8A9PV1O,A1047EDJ84IMAS,A105S56ODHGJEK,...,AZMY6E8B52L2T,AZNUHQSHZHSUE,AZO8OJ7YGSJSS,AZQAEL5GPBNIR,AZQGJ5CEAJGXB,AZQJ85BTRUQV2,AZU2JDR2GVICQ,AZYJE40XW6MFG,AZZ5ASC403N74,AZZD30PYJVGI7
asin,,,,,,,,,,,,,,,,,,,,,
B00000JBLH,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B00000JBLU,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B00000JBNX,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B00000JBO8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B00000JFNV,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B00000JZKB,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B00000K3T0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B00001MXVQ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
B00002NDS2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
from scipy.spatial.distance import correlation
neighbor = NearestNeighbors(n_neighbors=4, algorithm='auto', metric = 'correlation').fit(train_data_matrix)
distances, indices = neighbor.kneighbors(train_data_matrix)
print(distances)
print("__________________________")
print(indices)

[[0.00000000e+00 6.68894444e-01 6.72673165e-01 7.16357961e-01]
 [1.11022302e-16 9.01307246e-01 9.18604307e-01 9.52695273e-01]
 [0.00000000e+00 7.39125403e-01 9.48417962e-01 9.50309601e-01]
 ...
 [0.00000000e+00 7.13554050e-01 8.48792127e-01 8.90891055e-01]
 [0.00000000e+00 7.46679801e-01 7.49127397e-01 8.67017132e-01]
 [0.00000000e+00 5.00000000e-01 6.27322004e-01 6.59793091e-01]]
__________________________
[[   0  557 1259 1509]
 [   1  764 1582  229]
 [   2 1416  133 1703]
 ...
 [2417 1511 1388 1627]
 [2418 1271  992 1898]
 [2419 2159 2398 2016]]


In [17]:
# from sklearn.metrics.pairwise import pairwise_distances
# item_similarity = pairwise_distances(train_data_matrix, metric='cosine')
# item_similarity

In [18]:
reference_index = train_data_matrix.index.values
reference_index

array(['B00000JBLH', 'B00000JBLU', 'B00000JBNX', ..., 'B00KOO594O',
       'B00KOO599O', 'B00KYA0RC2'], dtype=object)

In [19]:
# train_data_matrix_Tr = train_data_matrix.T
# train_data_matrix_Tr.shape
# # item_similarity.shape

In [20]:
# item_prediction = train_data_matrix_Tr.dot(item_similarity)/(np.abs(item_similarity).sum(axis=1))
# item_prediction
# # item_prediction = item_prediction
# print(trainSet[1:2])
# print(trainSet[1:4]['reviewerID'].iat[0])
# print(trainSet[1:4]['asin'].iat[0])

for line in trainSet.values[:5]:
    print(line)

['B002FGUDBS' 'ATFBVUXDIRXT6' 5]
['B005IMAKYG' 'A3H7NPNDMGLOU4' 5]
['B0010JEJPC' 'AMW1OYXWOGZGJ' 5]
['B0085IQ28I' 'A15T9G38F589KM' 2]
['B009ZMWHIC' 'A32E5G5T902WRV' 1]


In [24]:
pred_rating = {}

for line in testSet.values[:500]:
    myuser = line[1]
    myitem = line[0]
    myrate = line[2]
# myuser = 'AMCAID3LTHKEC'   #testSet[1:4]['reviewerID'].iat[0]
# myitem = 'B00007LVCN'  #testSet[5:6]['asin'].iat[0]

    its_location, =np.where(reference_index == myitem)
    its_location = its_location[0]

    myitem_distances = distances[its_location:its_location+1][0]
    myitem_indices = indices[its_location:its_location+1][0]
    
    
    bi = mean_items.loc[myitem]['overall'] - mean_overall
    bx = mean_users.loc[myuser]['overall'] - mean_overall
    bxi = mean_overall + bi + bx

#     print(its_location)
#     print(myitem_distances)
#     print(myitem_indices)

#     print("____________________________________")

    item_occurences_in_train = []
    for i in range(4):
        item_occurence = trainSet.loc[(trainSet['asin'] == reference_index[myitem_indices[i]]) & (trainSet['reviewerID'] == myuser)]
        bi_n = mean_items.loc[reference_index[myitem_indices[i]]]['overall'] - mean_overall
        bix_n = mean_overall + bi_n + bx
        item_occurences_in_train.append((item_occurence, myitem_distances[i], bix_n))

#     print(item_occurences_in_train)

#     print("____________________________________")

    
#     print(bxi)

#     print("____________________________________")

    sim_rating = 0
#     divide_rating = 0
    # pred_rating((myuser,myitem)) = bxi + sim_rating

    for line in item_occurences_in_train:
#         print(line[0]['overall'].values)
        if not line[0].empty:
#             print(line[2])
            sim_rating = sim_rating + (line[0]['overall'].values-line[2])*(line[1])
            sim_rating=sim_rating[0]
#             print(sim_rating)
    #         divide_rating += (1-line[1]) 
#     print(sim_rating)

#     print("____________________________________")

    pred = bxi + sim_rating
    pred_rating[(myitem, myuser, myrate)] = pred

y_true = []
y_pred = []
for key in pred_rating:
    y_true.append(key[2])
    y_pred.append(pred_rating[key])
#     print(key," __ ", pred_rating[key])
# y_pred.append(pred_rating.values())
# print(y_pred[0])

mse = mean_squared_error(y_true, y_pred, multioutput='uniform_average')
print(mse)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [55]:

# print(item_occurences_in_train[0][0].overall)
# trainSet['reviewerID']

In [56]:
# bi = mean_items.loc[myitem]['overall'] - mean_overall
# bi

In [57]:
# bx = mean_users.loc[myuser]['overall'] - mean_overall
# bx

In [58]:
# bxi = mean_overall + bi + bx
# bxi

In [59]:
# pred_rating = {}
# sim_rating = 0
# divide_rating = 0
# # pred_rating((myuser,myitem)) = bxi + sim_rating

# for line in item_occurences_in_train:
#     if not line[0].empty:
# #         print(line[0]['overall'])
#         sim_rating += (line[0]['overall'])*(line[1])
# #         divide_rating += (1-line[1]) 
# sim_rating = sim_rating
# sim_rating

In [60]:
# pred = bxi + sim_rating
# pred_rating[(myuser,myitem)] = pred
# pred_rating

In [7]:
# users_get_item = input()

# for line in testSet.values:
#     this_item = line[0]
#     its_user = line[1]
#     its_rating = line[2]

In [8]:
df_clean_matrix = df_clean.pivot(index='asin', columns='reviewerID', values='overall').fillna(0)
df_clean_matrix = df_clean_matrix.T
R = (df_clean_matrix).as_matrix()
R

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [9]:
user_ratings_mean = np.mean(R, axis = 1)
R_demeaned = R - user_ratings_mean.reshape(-1, 1)
R_demeaned

array([[-0.00785124, -0.00785124, -0.00785124, ..., -0.00785124,
        -0.00785124, -0.00785124],
       [-0.01033058, -0.01033058, -0.01033058, ..., -0.01033058,
        -0.01033058, -0.01033058],
       [-0.01446281, -0.01446281, -0.01446281, ..., -0.01446281,
        -0.01446281, -0.01446281],
       ...,
       [-0.01983471, -0.01983471, -0.01983471, ..., -0.01983471,
        -0.01983471, -0.01983471],
       [-0.02272727, -0.02272727, -0.02272727, ..., -0.02272727,
        -0.02272727, -0.02272727],
       [-0.00991736, -0.00991736, -0.00991736, ..., -0.00991736,
        -0.00991736, -0.00991736]])

In [10]:
from scipy.sparse.linalg import svds
U, sigma, Vt = svds(R_demeaned)

In [13]:
sigma = np.diag(sigma)

In [14]:
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)
preds_df = pd.DataFrame(all_user_predicted_ratings, columns = df_clean_matrix.columns)
preds_df['reviewerID'] = users_df
preds_df.set_index('reviewerID', inplace=True)
preds_df

asin,B00000JBLH,B00000JBLU,B00000JBNX,B00000JBO8,B00000JFNV,B00000JZKB,B00000K3T0,B00001MXVQ,B00002NDS2,B00002QUKW,...,B00JXLGESY,B00JXLGETI,B00JXLGETS,B00JXLGEUC,B00JXLGF06,B00K5UZP2S,B00KLLBEPI,B00KOO594O,B00KOO599O,B00KYA0RC2
reviewerID,,,,,,,,,,,,,,,,,,,,,
A00473363TJ8YSZ3YAGG9,0.009418,0.009833,0.008133,0.009800,0.009255,0.009515,0.009459,0.009715,0.009519,0.009578,...,0.002120,0.005570,0.006253,0.005047,0.002059,0.009664,0.008558,0.007990,0.008169,0.009147
A04324702R5O7JRSQNVAH,0.001954,0.009015,0.012244,0.007353,0.052912,0.005420,0.005854,0.007832,0.010231,0.009903,...,0.014470,0.023122,-0.007040,0.010388,0.008466,0.012211,0.006726,0.008263,0.007641,0.013414
A0678500JUN3N7KUG4PS,0.015533,0.016497,0.016022,0.016520,0.019748,0.014926,0.015304,0.015718,0.015633,0.017690,...,0.016352,0.014596,0.010168,0.018504,0.019116,0.014860,0.017046,0.011634,0.016318,0.019699
A100UD67AHFODS,0.001849,0.008544,0.013974,0.007984,0.039157,0.005880,0.006967,0.008694,0.010729,0.011352,...,0.018061,0.023611,-0.006925,0.009560,0.014101,0.011629,0.007426,0.010582,0.010268,0.014981
A100WO06OQR8BQ,0.039053,0.041581,0.054373,0.023586,0.216318,0.027718,0.020682,0.014191,0.020580,0.029777,...,0.230772,0.136011,0.089123,0.246942,0.268509,0.022435,0.085529,0.021466,0.060942,0.076632
A100ZQDV7L8PVV,0.006844,0.003561,0.003778,0.004035,0.024764,0.004276,0.003495,0.003819,0.005412,0.002472,...,0.011034,0.008914,0.012440,0.006543,-0.001894,0.006667,0.002643,0.000819,-0.000863,-0.002492
A101PIL0I3UZJJ,0.009101,0.009268,0.008157,0.009546,0.007181,0.009119,0.009196,0.009431,0.009261,0.009536,...,0.003445,0.005652,0.006004,0.005049,0.003243,0.009152,0.008357,0.007558,0.008123,0.009002
A1027EV8A9PV1O,0.021102,0.021661,0.020561,0.022217,0.013482,0.021478,0.021792,0.021967,0.021459,0.022733,...,0.014642,0.016321,0.015730,0.016814,0.016981,0.020791,0.021256,0.019553,0.021449,0.023077
A1047EDJ84IMAS,0.012319,0.012352,0.010700,0.012729,0.009229,0.012289,0.012307,0.012557,0.012324,0.012591,...,0.004439,0.007304,0.008418,0.006823,0.004007,0.012217,0.011208,0.010005,0.010683,0.011730


In [49]:
def recommend_method(predictions_df, movies_df, original_ratings_df, num_recommendations=5):
    
    # Get and sort the user's predictions
    sorted_user_predictions = predictions_df.loc['AZ9WQM7SLUX7E'].sort_values(ascending=False)
    
    # Get the user's data and merge in the movie information.
    user_data = original_ratings_df[original_ratings_df.reviewerID == 'AZ9WQM7SLUX7E']
    user_full = (user_data.merge(movies_df, how = 'left', left_on = 'asin', right_on = 'asin').
                     sort_values(['overall'], ascending=False)
                 )

    print ('User {0} has already rated {1} items.'.format('AZ9WQM7SLUX7E', user_full.shape[0]))
    print ('Recommending the highest {0} predicted ratings for items not already rated.'.format(num_recommendations))
    
    # Recommend the highest predicted rating movies that the user hasn't seen yet.
    recommendations = (movies_df[~movies_df['asin'].isin(user_full['asin'])].
         merge(pd.DataFrame(sorted_user_predictions).reset_index(), how = 'left',
               left_on = 'asin',
               right_on = 'asin').
         rename(columns = {'AZ9WQM7SLUX7E': 'Predictions'}).
         sort_values('Predictions', ascending = False).
                       iloc[:num_recommendations, :-1]
                      )

    return sorted_user_predictions

In [51]:
already_rated = recommend_method(preds_df, items_df, df_clean, 10)

User AZ9WQM7SLUX7E has already rated 21 items.
Recommending the highest 10 predicted ratings for items not already rated.


In [55]:
already_rated

asin
B0010T3QT2    3.077894
B0039N7ELS    2.457829
B0035FX5MC    1.505538
B0009F3P3U    1.412224
B0039N3QFQ    1.346022
B0000538AC    1.292004
B003FHBPRM    1.183063
B002K9IHJK    1.167714
B0039N3QO2    1.120583
B003FHF98S    1.109153
B000WU4H5C    1.068803
B002K9M7MS    0.971898
B0027CTFBO    0.946557
B002ECFIDG    0.928427
B000XSPEPA    0.818832
B000MFHX3U    0.745484
B004O49F7M    0.721668
B003H04A42    0.710097
B0039N7EI6    0.698696
B001BM2M8Y    0.650492
B002ECFILI    0.640541
B000I0VMMC    0.636779
B004VJSFG2    0.606254
B000ZHB2HS    0.601527
B003BHLMMY    0.591913
B0026ICM1E    0.581051
B000FH1J6C    0.555350
B002K9PIKG    0.550495
B002XYO84U    0.545924
B0006SV7Q2    0.530230
                ...   
B002ONCDH4   -0.025957
B00000JZKB   -0.026226
B001UQ6E58   -0.026958
B000HHS8C8   -0.028745
B001PMJXYG   -0.029076
B005OKRKRW   -0.030678
B002FGUDDG   -0.031169
B005ZOCWYI   -0.031726
B0019W4EIW   -0.034319
B004HY9IJE   -0.035263
B005GQKNL4   -0.037258
B004H3XKR6   -0.038644
B00FZ9

In [42]:
predictions

,asin
1440,B0039N7ELS
450,B0009F3P3U
1467,B003FHBPRM
1283,B002K9IHJK
1438,B0039N3QO2
1468,B003FHF98S
788,B000WU4H5C
1286,B002K9M7MS
1207,B0027CTFBO
799,B000XSPEPA
